# Alpaca Benro Polaris - Data5 Telemetary Plot

This notebook visualizes telemetry from the Benro Polaris mount as well as information from the PID Controller that has been recorded into the `alpaca.csv` log file. It shows the delta_ref (Ra/Dec/PA), the alpha_ref (Az,Alt,Roll), the theta_ref (theta1,theta2,theta3), the theta_meas (theta1,theta2,theta3), the omega_op (omega1,omega2,omega3).

To ensure compatible data is captured for this notebook, update the `config.toml` file with the following setting:

```toml
log_performance_data = 6
```

After making this change, restart the Alpaca Driver to begin logging the required telemetry. The `alpaca.csv` should now include lines with the following type of data.
```
2025-08-09T07:17:55.971 INFO ,Dataset,  Mode,  DRef1,DRef2,DRef3, ARef1,ARef2,ARef3, TRef1,TRef2,TRef3, TMeas,TMeas2,TMeas3, ORef1,ORef2,ORef3
2025-08-09T07:17:55.972 INFO ==STARTUP== ALPACA BENRO POLARIS DRIVER v1.0.2 =========== 
2025-08-09T07:17:56.410 INFO ,"DATA6", "AUTO",  138.1050,-66.9609,94.4666,  207.7991,34.5164,-1.9723,  204.3214,34.5657,2.8649,  204.3214,34.5657,2.8649,  0.0000,0.0000,0.0000
2025-08-09T07:17:56.628 INFO ,"DATA6", "IDLE",  138.1060,-66.9606,94.4669,  207.7995,34.5163,-1.9722,  204.3218,34.5657,2.8649,  204.3218,34.5657,2.8649,  0.0000,0.0000,0.0000
2025-08-09T07:17:56.835 INFO ,"DATA6", "IDLE",  138.1072,-66.9607,94.4665,  207.7993,34.5165,-1.9723,  204.3215,34.5658,2.8650,  204.3215,34.5658,2.8650,  0.0000,0.0000,0.0000
```
Timestamp & Metadata
- **Timestamp**: ISO 8601 format (e.g. `2025-07-24T09:48:05.500`) — precise time of the log entry.
- **INFO**: Log level indicator.
- **DATA6**: Tag identifying this line as a telemetry record relevant to the DATA6 format.

Reference Equatorial Orientation (RA, Dec, PA) based on Goto target
- **RA**: Right ascension (degrees) — Celestrial co-ordinate is analogous to longitude on Earth,
- **Dec**: Declination (degrees) — Celestrial co-ordinate is similar to latitude. 
- **PA**: Parallactic Angle (degrees) — Equatorial Frame Rotation (0=Down points to celestrial pole, +ve CCW, -ve CW)

Refence AzAlt Orientation (Azimuth, Altitude, Roll) based on Goto target or tracking target
- **az**: Azimuth angle (degrees) — horizontal rotation relative to true north.
- **alt**: Altitude angle (degrees) — vertical angle above the horizon.
- **roll**: Roll angle (degrees) — rotation around the camera boresight or optical axis.

Motor Angular Position Refenence (theta_ref) based on Quaternion1 conversion of alpha_ref
- **theta1, theta2, theta3**: Angular positions for each motor axis (degrees). 

Motor Angular Velocity Control Signal Outputs (omega_op) based on pid controller
- **omega1, omega2, omega3**: Angular velocities (degrees/sec) for each motor axis.


In [3]:
import sys
sys.path.insert(0, '../driver')

In [4]:
import plotly.graph_objects as go
import pandas as pd
from plotly.subplots import make_subplots
from control import polar_rotation_angle

In [13]:
import pandas as pd
import csv



def read_data6_logfile(filename):
    columns = [
        "mode",
        "dref1", "dref2", "dref3",        # RA, Dec, PA
        "aref1", "aref2", "aref3",        # Az, Alt, Roll
        "tref1", "tref2", "tref3",        # Theta_ref
        "tmeas1", "tmeas2", "tmeas3",     # Theta_meas
        "omega1", "omega2", "omega3"      # Omega_op
    ]


    # Read and parse only DATA5 lines
    data = []
    with open(filename) as f:
        for line in f:
            if "DATA6" in line:
                fields = line.strip().split(",")
                if len(fields) >= 10:
                    values = [fields[2]] + [float(val) for val in fields[3:]]  # Skip timestamp and DATA5 label
                    data.append(values)

 
    df = pd.DataFrame(data, columns=columns)

    return df


def plot_data6_theta_omega(df):
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=[
            "Theta1: Ref vs Meas", "Omega1: Output",
            "Theta2: Ref vs Meas", "Omega2: Output",
            "Theta3: Ref vs Meas", "Omega3: Output"
        ],
        shared_xaxes=True,
        vertical_spacing=0.1
    )

    for i in range(3):
        n = i + 1
        # Theta subplot (left column)
        fig.add_trace(go.Scatter(
            y=df[f'tref{n}'],
            mode="lines",
            name=f'Theta{n} Ref',
            line=dict(color="orange")
        ), row=n, col=1)

        fig.add_trace(go.Scatter(
            y=df[f'tmeas{n}'],
            mode="markers",
            name=f'Theta{n} Meas',
            marker=dict(color="blue", size=6)
        ), row=n, col=1)

        # Omega subplot (right column)
        fig.add_trace(go.Scatter(
            y=df[f'omega{n}'],
            mode="lines+markers",
            name=f'Omega{n} Output',
            line=dict(color="green"),
            marker=dict(size=5)
        ), row=n, col=2)

    fig.update_layout(
        height=1000,
        width=1200,
        title="DATA6: Theta Reference vs Measured and Omega Output",
        xaxis6_title="Time (frame index)",
        legend_title_text="Signal",
        showlegend=True
    )

    fig.update_yaxes(title_text="Theta (deg)", row=1, col=1)
    fig.update_yaxes(title_text="Omega (deg/sec)", row=1, col=2)
    fig.update_yaxes(title_text="Theta (deg)", row=2, col=1)
    fig.update_yaxes(title_text="Omega (deg/sec)", row=2, col=2)
    fig.update_yaxes(title_text="Theta (deg)", row=3, col=1)
    fig.update_yaxes(title_text="Omega (deg/sec)", row=3, col=2)

    fig.show()




In [ ]:
filename = "../logs/alpaca.csv.1"

df = read_data6_logfile(filename)

plot_data6_theta_omega(df)

2025-08-09T07:17:55.971 INFO ,Dataset,  Mode,  DRef1,DRef2,DRef3, ARef1,ARef2,ARef3, TRef1,TRef2,TRef3, TMeas,TMeas2,TMeas3, ORef1,ORef2,ORef3

2025-08-09T07:17:55.972 INFO ==STARTUP== ALPACA BENRO POLARIS DRIVER v1.0.2 =========== 

2025-08-09T07:17:55.987 INFO ==STARTUP== Serving Stellarium Telescope on :10001

2025-08-09T07:17:56.108 INFO ==STARTUP== Serving ASCOM Alpaca on :5555. Time stamps are UTC.

2025-08-09T07:17:56.177 INFO ==STARTUP== Starting Polaris Client on 192.168.0.1:9090. 

2025-08-09T07:17:56.177 INFO Polaris communication init...

2025-08-09T07:17:56.178 INFO ->> Polaris: MODE query status info request

2025-08-09T07:17:56.182 INFO <<- Polaris: response to command received: 284 mode:8;state:0;track:0;speed:0;halfSpeed:0;remNum:;runTime:;photoNum:;pause:;interval:;repeNum:;

2025-08-09T07:17:56.193 INFO Polaris communication init... done

2025-08-09T07:17:56.193 INFO Site lat = -33.8598874 (-033d51'35.59") | lon = 151.2021771 (+151d12'07.84").

2025-08-09T07:17:56.19

ValueError: could not convert string to float: ' "AUTO"'